# Ejercicios Pair Programming - Modulo 2 - Sprint 2 
## Lección 3 - ETL III: Transformación II, Clases y Funciones de limpieza
### Ana Gonzalez y Sonia Ruiz

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

**Happy coding** 🦈

In [2]:
#Importamos las librerias necesarias para poder trabajar en este archivo de jupyter.
import pandas as pd
import numpy as np
import requests

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Creamos la clase 'Clima' para trabajar con los datos que obtenemos de la API de '7timer'.
class Clima:
    def __init__(self): #creamos el constructor, en este caso no le ponemos ninguna variable global.
    
    def obtener_prediccion(self,dict_countries,product):    #Funcion para obtener datos meteorologicos del tipo
                                                            # product que le indiquemos.
        #El parametro "dict_countries", debe ser un diccionario donde cada key sera un pais y sus values
        #  seran una lista con la latitud y la longitud (en ese orden).
        #El parametro "product" sera a igual a 'civil', 'civillight', 'astro' o 'meteo'.

        df_countries=pd.DataFrame() #creamos un dataframe vacio.

        for key, value in dict_countries.items():   #Hacemos un for loop para obtener los datos meteorologicos
                                                    # de cada pais del diccionario.
            print(key, value)
            url = f'http://www.7timer.info/bin/api.pl?lon={value[1]}&lat={value[0]}&product=meteo&output=json'
            response=requests.get(url=url)
            code=response.status_code
            reason=response.reason
            
            df=pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
            df['country']=key
            df_countries=pd.concat([df_countries,df],axis=0,ignore_index=False)
        return df_countries
    
    def obtener_ataques(self,dataframe,lista_paises):
        #Funcion para obtener un dataframe con los ataques ocurridos solo en los paises que aparecen en
        # la lista indicada como parametro "lista_paises". En "dataframe" introduciremos el la variable que
        # almacena el dataframe con los ataque sproducidos en todo el mundo.
        df_ataques_paises=dataframe[dataframe['country'].isin(lista_paises)]
        print('El nuevo dataframe contiene los ataques ocurridos en',df_ataques_paises['country'].unique())
        return df_ataques_paises
    
    def separar_columna_rh(self,dataframe):
        #Funcion para separar una columna que contenga un alista de diccionarios en varias columnas por separado.
        #"dataframe", sera el dataframe donde se encuentra la columna que queremos separar.

        df_rh=dataframe['rh_profile'].apply(pd.Series)

        #Separamos la columna, para ello hacemos un for para iterar por cada una de las columnas del dataframe
        #  generado a partir de la columna que queriamos separar. 
        for i in range(len(df_rh.columns)): 

            #Aplicamos el apply,extraemos el valores de la key "layer" y lo almacenamos en una variable que
            #  convertimos a string.
            nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

            #Hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda.
            valores = list(df_rh[i].apply(pd.Series)["rh"] )

            #Usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe
            #  con la información del clima. 
            dataframe.insert(i, nombre, valores)
            dataframe.drop(['rh_profile'],inplace=True,axis=1)
            dataframe=dataframe.reset_index()
            dataframe.drop(['index'],inplace=True,axis=1)
        return dataframe

    def separar_columna_wind(self,dataframe):
        #Funcion para separar una columna que contenga un alista de diccionarios en varias columnas por separado.
        #"dataframe", sera el dataframe donde se encuentra la columna que queremos separar.

        df_wind=dataframe['wind_profile'].apply(pd.Series)

        #Separamos la columna "wind_profile", para ello hacemos un for para iterar por cada una de las columnas. 
        for i in range(len(df_wind.columns)): 

            #Aplicamos el apply,extraemos el valores de la key "layer" y lo almacenamos en una variable
            #  que convertimos a string 
            nombre = "wind_dir_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
            nombre2 = "wind_spe_" + str(df_wind[i].apply(pd.Series)["layer"][0])

            #Hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda.
            direccion = list(df_wind[i].apply(pd.Series)["direction"])

            velocidad= list(df_wind[i].apply(pd.Series)["speed"])

            #Usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe
            #  con la información del clima. 
            dataframe.insert(i, nombre, direccion)
            dataframe.insert(i, nombre2, velocidad)
            dataframe.drop(['wind_profile'],inplace=True,axis=1)
            dataframe=dataframe.reset_index()
            dataframe.drop(['index'],inplace=True,axis=1)
        return dataframe
    
    def juntar_dataframes(self,dataframe_tiempo,dataframe_ataques):
        #Funcion para juntar dos dataframes, el dataframe que contiene los datos de los ataques y el dataframe
        #  que contiene la informacion climatica.

        #Hacemos el groupby por pais.
        df_agrupado=dataframe_tiempo.groupby('country').mean()
        df_agrupado.reset_index()
        df_clima3=df_agrupado.drop(['index'],inplace=False,axis=1)
        df_clima3.reset_index(inplace=True)
        df_clima3['country']=df_clima3['country'].str.lower()
        df_mergeado=pd.merge(left=dataframe_ataques,right=df_clima3,how='left',left_on='country',right_on='country')
        return df_mergeado

    def guardar_dataframe(self,tipo_fichero,dataframe_guardar,direccion_guardado):
        #"tipo_fichero", especificaremos si queremos guardarlo en csv o pickle, debera ir entre comillas.
        #"dataframe_guardar", pondremos la variable que almacena el dataframe que queremos almacenar en un fichero.
        #"direccion_guardado", indicaremos la ruta donde lo vamos a almacenar, debera ir entre comillas.
        if tipo_fichero=='.csv':
            dataframe_guardar.to_csv(direccion_guardado)
            print('Dataframe almacenado como .csv exitosamente en la ubicacion',direccion_guardado)
        elif tipo_fichero=='.pkl':
            dataframe_guardar.to_pickle(direccion_guardado)
            print('Dataframe almacenado como .pkl exitosamente en la ubicacion',direccion_guardado)
        else:
            print('La funcion solo acepta como tipos de fichero ".csv" o ".pkl".')
        return 'Funcion guardar_dataframe finalizada.'


In [ ]:
api=Clima(lat,lon,country)
api